In [1]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency

In [2]:
df = pd.read_csv('DATA/train_sample500.csv')

In [3]:
df['ip']=df['ip'].astype('category')
df['app']=df['app'].astype('category')
df['device']=df['device'].astype('category')
df['channel']=df['channel'].astype('category')
df['os']=df['os'].astype('category')

In [4]:
#pd.set_option('display.max_rows', None)

df.head(1000).sort_values(by='attributed_time', ascending=False)

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,8645,2,1,35,469,2017-11-08 05:00:59,NaN,0
1,102065,2,1,30,237,2017-11-08 21:03:18,NaN,0
2,284766,12,1,19,178,2017-11-08 01:07:37,NaN,0
3,43289,15,1,17,315,2017-11-09 08:25:59,NaN,0
4,267426,12,1,16,340,2017-11-08 08:19:49,NaN,0
...,...,...,...,...,...,...,...,...
995,2765,208,1,19,330,2017-11-08 00:08:41,NaN,0
996,3564,2,1,11,469,2017-11-08 06:29:33,NaN,0
997,25761,1,2,9,125,2017-11-08 14:25:16,NaN,0
998,80532,5,1,17,377,2017-11-08 14:59:15,NaN,0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   ip               500000 non-null  category
 1   app              500000 non-null  category
 2   device           500000 non-null  category
 3   os               500000 non-null  category
 4   channel          500000 non-null  category
 5   click_time       500000 non-null  object  
 6   attributed_time  1257 non-null    object  
 7   is_attributed    500000 non-null  int64   
dtypes: category(5), int64(1), object(2)
memory usage: 20.2+ MB


In [6]:
df['ip'].value_counts()

5348      3516
5314      3154
73516     2067
73487     2040
53454     1342
          ... 
198810       1
65969        1
198782       1
198746       1
129192       1
Name: ip, Length: 66138, dtype: int64

In [7]:
df['device'].value_counts()

1       471432
2        21979
0         2748
3032      1908
3543       726
         ...  
259          1
249          1
242          1
241          1
4001         1
Name: device, Length: 267, dtype: int64

In [8]:
fraud_clicks = df['is_attributed']==0

In [9]:
fraud_clicks

0         True
1         True
2         True
3         True
4         True
          ... 
499995    True
499996    True
499997    True
499998    True
499999    True
Name: is_attributed, Length: 500000, dtype: bool

In [10]:
fraud_clicks.value_counts()

True     498743
False      1257
Name: is_attributed, dtype: int64

In [11]:
df_fraud_clicks = df[fraud_clicks]

In [12]:
df_fraud_clicks

,ip,app,device,os,channel,click_time,attributed_time,is_attributed
0,8645,2,1,35,469,2017-11-08 05:00:59,NaN,0
1,102065,2,1,30,237,2017-11-08 21:03:18,NaN,0
2,284766,12,1,19,178,2017-11-08 01:07:37,NaN,0
3,43289,15,1,17,315,2017-11-09 08:25:59,NaN,0
4,267426,12,1,16,340,2017-11-08 08:19:49,NaN,0
...,...,...,...,...,...,...,...,...
499995,123994,6,1,37,459,2017-11-08 14:52:44,NaN,0
499996,20556,3,1,19,280,2017-11-07 09:09:07,NaN,0
499997,194075,12,1,18,140,2017-11-07 13:37:47,NaN,0
499998,26995,1,1,10,115,2017-11-08 11:05:06,NaN,0


In [13]:
df_fraud_clicks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498743 entries, 0 to 499999
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype   
---  ------           --------------   -----   
 0   ip               498743 non-null  category
 1   app              498743 non-null  category
 2   device           498743 non-null  category
 3   os               498743 non-null  category
 4   channel          498743 non-null  category
 5   click_time       498743 non-null  object  
 6   attributed_time  0 non-null       object  
 7   is_attributed    498743 non-null  int64   
dtypes: category(5), int64(1), object(2)
memory usage: 24.0+ MB


In [14]:
df2 = df_fraud_clicks[['ip','device']]

In [15]:
# df of fraudulent clicks by ip and device
df2

,ip,device
0,8645,1
1,102065,1
2,284766,1
3,43289,1
4,267426,1
...,...,...
499995,123994,1
499996,20556,1
499997,194075,1
499998,26995,1


In [16]:
#pd.crosstab(df_fraud_clicks['ip'], df_fraud_clicks['device']).sort_values(by='device',ascending=False).head(20)

In [17]:
#df.groupby('Outlet_Location_Type').agg([np.mean, np.median])[['Item_Outlet_Sales', 'Item_Visibility']]
ip_cnt = df2.groupby('ip').count()[['device']]

In [18]:
ip_cnt.sort_values(by='device', ascending=False)

,device
ip,
5348,3511
5314,3145
73516,2064
73487,2034
53454,1342
...,...
348405,0
359317,0
325608,0


In [19]:
ip_cnt.sort_values(by='ip',ascending=True)

,device
ip,
6,4
9,10
10,2
19,2
20,15
...,...
364721,1
364744,2
364757,2


In [20]:
ip_cnt.to_csv(r'/Users/galvanize/Documents/capstone3/ip_device.csv')

In [21]:
#df2[mask]

In [22]:
df2['ip'].value_counts()

5348      3511
5314      3145
73516     2064
73487     2034
53454     1342
          ... 
350089       0
236329       0
272592       0
216176       0
164805       0
Name: ip, Length: 66138, dtype: int64

In [23]:
df2['device'].value_counts()

1       470575
2        21971
0         2469
3032      1908
3543       726
         ...  
179          0
132          0
131          0
118          0
4001         0
Name: device, Length: 267, dtype: int64

In [24]:
t = df2['device'].value_counts()

In [25]:
t

1       470575
2        21971
0         2469
3032      1908
3543       726
         ...  
179          0
132          0
131          0
118          0
4001         0
Name: device, Length: 267, dtype: int64

In [26]:
pd.crosstab(df2['ip'], df2['device'])

device,0,1,2,4,5,6,7,8,9,11,...,3043,3121,3158,3164,3409,3543,3545,3573,3866,3867
ip,,,,,,,,,,,,,,,,,,,,,
6,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,15,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
364721,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
364744,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
364757,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 498743 entries, 0 to 499999
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   ip      498743 non-null  category
 1   device  498743 non-null  category
dtypes: category(2)
memory usage: 9.7 MB


In [28]:
observation = pd.crosstab(df2['ip'], df2['device'])

In [29]:
chi2, p, dof, expected = chi2_contingency(observation.values)

print(f'The test statistic is {chi2}. \nThe p value is {p}.')

The test statistic is 30192368.322958387. 
The p value is 0.0.


N0: The